In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI管道：使用google-cloud-pipeline-components训练、上传和部署TPU模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/official/pipelines/google_cloud_pipeline_components_TPU_model_train_upload_deploy.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/notebooks/blob/master/official/pipelines/google_cloud_pipeline_components_TPU_model_train_upload_deploy.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/pipelines/google_cloud_pipeline_components_TPU_model_train_upload_deploy.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI工作台中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本笔记本展示如何使用[`google_cloud_pipeline_components`](https://github.com/kubeflow/pipelines/tree/master/components/google-cloud)中定义的组件结合实验性的`run_as_aiplatform_custom_job`方法，构建一个[Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines)工作流程，用于训练使用TPUs的[自定义模型](https://cloud.google.com/vertex-ai/docs/training/containers-overview)，上传模型作为`Model`资源，创建`Endpoint`资源，并将`Model`资源部署到`Endpoint`资源中。

了解更多关于[使用TPU加速器进行训练](https://cloud.google.com/vertex-ai/docs/training/training-with-tpu-vm)。

### 数据集

本教程使用的数据集是来自[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/overview)的[cifar10数据集](https://www.tensorflow.org/datasets/catalog/cifar10)。您将使用的数据集版本已内置于TensorFlow中。训练的模型可预测图像属于十个类别中的哪一类：飞机、汽车、鸟、猫、鹿、狗、青蛙、马、船或卡车。

### 目标

在本教程中，您将学习如何使用来自`google_cloud_pipeline_components`的组件和您构建的自定义管道组件来创建自定义模型的流水线。

本教程使用以下谷歌云ML服务：

- `Vertex AI 训练`
- `Vertex AI 管道`
- `Google Cloud Pipeline Components`

执行的步骤包括：

- 为自定义模型构建自定义容器。
- 使用TPU训练自定义模型。
- 将训练好的模型上传为`Model`资源。
- 创建一个`Endpoint`资源。
- 将`Model`资源部署到`Endpoint`资源上。

成本

本教程使用Google Cloud的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI 的定价](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 的定价](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/) 根据您的预期使用量生成成本估算。

## 安装

安装执行此笔记本所需的软件包。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-storage \
                                 google-cloud-pipeline-components \
                                 kfp

### 仅限Colab使用：取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 开始之前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作:
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 参阅支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID} --quiet

#### 区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 验证您的谷歌云帐户

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关指示操作。

**1. Vertex AI Workbench**
* 不需要做任何操作，因为您已经通过身份验证。

**2. 本地JupyterLab实例，请取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

查看如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples为您的服务账户授予Cloud Storage权限。

创建一个云存储桶

创建一个存储桶来存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有当您的存储桶尚不存在时，才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

#### 服务账号

**如果你不知道你的服务账号**，尝试使用`gcloud`命令通过执行下面的第二个单元格来获取你的服务账号。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

为Vertex AI Pipelines设置服务账号访问权限

运行以下命令，将您的服务账号授权以读取和写入在上一步中创建的存储桶中的管道工件--每个服务账号只需运行一次。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。

### 导入库并定义常量

In [ ]:
import os
from typing import Any, Dict, List

import google.cloud.aiplatform as aip
import kfp
from google_cloud_pipeline_components.types import artifact_types
from google_cloud_pipeline_components.v1.custom_job.component import \
    custom_training_job as CustomTrainingJobOp
from google_cloud_pipeline_components.v1.endpoint import (EndpointCreateOp,
                                                          ModelDeployOp)
from google_cloud_pipeline_components.v1.model import ModelUploadOp
from kfp import compiler
from kfp.dsl import importer_node

设置以下 Vertex AI Pipelines 的常量：

In [ ]:
PIPELINE_ROOT = "{}/pipeline_root/tpu_cifar10_pipeline".format(BUCKET_URI)

## 初始化用于Python的Vertex AI SDK

为您的项目和相应的存储桶初始化用于Python的Vertex AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

### 设置变量

接下来，设置一些在教程中使用的变量。

#### 设置硬件加速器

您可以为训练和预测都设置硬件加速器。

将变量`TRAIN_TPU/TRAIN_NTPU`设置为使用支持TPU的容器训练镜像和分配的TPU数量，将`DEPLOY_GPU/DEPLOY_NGPU`设置为使用支持GPU的容器部署镜像和分配给虚拟机实例的GPU数量。

查看[可用加速器的位置](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

否则，指定`(None, None)`以使用在CPU上运行的容器映像。

In [ ]:
from google.cloud.aiplatform import gapic

TRAIN_TPU, TRAIN_NTPU = (
    gapic.AcceleratorType.TPU_V2,
    8,
)  # Using TPU_V2 with 8 accelerators

DEPLOY_GPU, DEPLOY_NGPU = (gapic.AcceleratorType.NVIDIA_TESLA_K80, 1)

#### 设置预构建容器

Vertex AI提供预构建容器来运行训练和预测。

有关最新列表，请参见[用于训练的预构建容器](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)和[用于预测的预构建容器](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers)。

In [ ]:
DEPLOY_VERSION = "tf2-gpu.2-9"

DEPLOY_IMAGE = "us-docker.pkg.dev/cloud-aiplatform/prediction/{}:latest".format(
    DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量`TRAIN_COMPUTE`和`DEPLOY_COMPUTE`来配置您的训练和预测的计算资源。
  - `机器类型`
     - `cloud-tpu`：用于TPU训练。请查看[TPU架构网站](https://cloud.google.com/tpu/docs/system-architecture-tpu-vm)了解详情
     - `n1-standard`：每个vCPU 3.75GB的内存
     - `n1-highmem`：每个vCPU 6.5GB的内存
     - `n1-highcpu`：每个vCPU 0.9GB的内存
  - `vCPUs`：\[2, 4, 8, 16, 32, 64, 96\]的数量

*注：以下内容不支持用于训练：*

   - `standard`：2个vCPUs
   - `highcpu`：2、4和8个vCPUs

*注：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
MACHINE_TYPE = "cloud-tpu"

# TPU VMs do not require VCPU definition
TRAIN_COMPUTE = MACHINE_TYPE
print("Train machine type", TRAIN_COMPUTE)

MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

In [ ]:
if not TRAIN_NTPU or TRAIN_NTPU < 2:
    TRAIN_STRATEGY = "single"
else:
    TRAIN_STRATEGY = "tpu"

EPOCHS = 20
STEPS = 10000

TRAINER_ARGS = [
    "--epochs=" + str(EPOCHS),
    "--steps=" + str(STEPS),
    "--distribute=" + TRAIN_STRATEGY,
]

# create working dir to pass to job spec
WORKING_DIR = f"{PIPELINE_ROOT}/model"

MODEL_DISPLAY_NAME = "tpu_train_deploy"
print(TRAINER_ARGS, WORKING_DIR, MODEL_DISPLAY_NAME)

创建一个自定义容器

我们将创建一个目录，并将所有容器构建产物写入该文件夹。

In [ ]:
CONTAINER_ARTIFACTS_DIR = "tpu-container-artifacts"

!mkdir {CONTAINER_ARTIFACTS_DIR}

写Dockerfile

In [ ]:
dockerfile = """FROM python:3.8

WORKDIR /root

# Copies the trainer code to the docker image.
COPY train.py /root/train.py

RUN pip3 install tensorflow-datasets

# Install TPU Tensorflow and dependencies.
# libtpu.so must be under the '/lib' directory.
RUN wget https://storage.googleapis.com/cloud-tpu-tpuvm-artifacts/libtpu/20210525/libtpu.so -O /lib/libtpu.so
RUN chmod 777 /lib/libtpu.so

RUN wget https://storage.googleapis.com/cloud-tpu-tpuvm-artifacts/tensorflow/20210525/tf_nightly-2.6.0-cp38-cp38-linux_x86_64.whl
RUN pip3 install tf_nightly-2.6.0-cp38-cp38-linux_x86_64.whl
RUN rm tf_nightly-2.6.0-cp38-cp38-linux_x86_64.whl

ENTRYPOINT ["python3", "train.py"]
"""

with open(os.path.join(CONTAINER_ARTIFACTS_DIR, "Dockerfile"), "w") as f:
    f.write(dockerfile)

#### 训练脚本

在下一个单元格中，您可以编写训练脚本 `train.py` 的内容。 总结如下：

- 从环境变量 `AIP_MODEL_DIR` 中获取保存模型工件的目录。 这个变量是由训练服务设置的。
- 从 TF 数据集（tfds）加载 CIFAR10 数据集。
- 使用 TF.Keras 模型 API 构建模型。
- 编译模型（`compile()`）。
- 根据参数 `args.distribute` 设置训练分布策略。
- 使用参数 `args.epochs` 和 `args.steps` 训练模型（`fit()`）。
- 将经过训练的模型保存（`save(MODEL_DIR)`）到指定的模型目录中。

下面列出了专门针对 TPU 的更改：
- 添加了一个部分，用于查找 TPU 集群，连接到它，并将训练策略设置为 TPUStrategy。
- 添加了一个部分，将经过训练的 TPU 模型保存到本地设备，以便可以保存到 AIP_MODEL_DIR 中。

In [ ]:
%%writefile {CONTAINER_ARTIFACTS_DIR}/train.py
# Single, Mirror and Multi-Machine Distributed Training for CIFAR-10

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--lr', dest='lr',
                    default=0.01, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print('DEVICES', device_lib.list_local_devices())

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple TPU devices
elif args.distribute == 'tpu':
    cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu="local")
    tf.config.experimental_connect_to_cluster(cluster_resolver)
    tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
    strategy = tf.distribute.TPUStrategy(cluster_resolver)
    print("All devices: ", tf.config.list_logical_devices('TPU'))
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))

# Preparing dataset
BUFFER_SIZE = 10000
BATCH_SIZE = 64

def make_datasets_unbatched():
  # Scaling CIFAR10 data from (0, 255] to (0., 1.]
  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image, label

  datasets, info = tfds.load(name='cifar10',
                            with_info=True,
                            as_supervised=True)
  return datasets['train'].map(scale).cache().shuffle(BUFFER_SIZE).repeat()


# Build the Keras model
def build_and_compile_cnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(32, 32, 3)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      optimizer=tf.keras.optimizers.SGD(learning_rate=args.lr),
      metrics=['accuracy'])
  return model

# Train the model
NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS
MODEL_DIR = os.getenv("AIP_MODEL_DIR")

train_dataset = make_datasets_unbatched().batch(GLOBAL_BATCH_SIZE)

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_cnn_model()

model.fit(x=train_dataset, epochs=args.epochs, steps_per_epoch=args.steps)
if args.distribute=="tpu":
    save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
    model.save(MODEL_DIR, options=save_locally)
else:
    model.save(MODEL_DIR)

### 建立容器

启用Artifact Registry API
您必须为您的项目启用Artifact Registry API服务。

<a href="https://cloud.google.com/artifact-registry/docs/enable-service">了解更多关于启用服务的信息</a>。

In [ ]:
! gcloud services enable artifactregistry.googleapis.com

if os.getenv("IS_TESTING"):
    ! sudo apt-get update --yes && sudo apt-get --only-upgrade --yes install google-cloud-sdk-cloud-run-proxy google-cloud-sdk-harbourbridge google-cloud-sdk-cbt google-cloud-sdk-gke-gcloud-auth-plugin google-cloud-sdk-kpt google-cloud-sdk-local-extract google-cloud-sdk-minikube google-cloud-sdk-app-engine-java google-cloud-sdk-app-engine-go google-cloud-sdk-app-engine-python google-cloud-sdk-spanner-emulator google-cloud-sdk-bigtable-emulator google-cloud-sdk-nomos google-cloud-sdk-package-go-module google-cloud-sdk-firestore-emulator kubectl google-cloud-sdk-datastore-emulator google-cloud-sdk-app-engine-python-extras google-cloud-sdk-cloud-build-local google-cloud-sdk-kubectl-oidc google-cloud-sdk-anthos-auth google-cloud-sdk-app-engine-grpc google-cloud-sdk-pubsub-emulator google-cloud-sdk-datalab google-cloud-sdk-skaffold google-cloud-sdk google-cloud-sdk-terraform-tools google-cloud-sdk-config-connector
    ! gcloud components update --quiet

请运行这些Artifact Registry和Docker步骤一次。

In [ ]:
!sudo usermod -a -G docker ${USER}

In [ ]:
REPOSITORY = "tpu-training-repository"
IMAGE = "tpu-train"

In [ ]:
!gcloud auth configure-docker us-central1-docker.pkg.dev --quiet

In [ ]:
!gcloud artifacts repositories create $REPOSITORY --repository-format=docker \
--location=us-central1 --description="Vertex TPU training repository"

构建训练图像

In [ ]:
TRAIN_IMAGE = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}:latest"

In [ ]:
print(TRAIN_IMAGE)

In [ ]:
%cd $CONTAINER_ARTIFACTS_DIR

In [ ]:
# Use quiet flag as the output is fairly large
!docker build --quiet \
    --tag={TRAIN_IMAGE} \
    .

In [ ]:
!docker push {TRAIN_IMAGE}

In [ ]:
%cd ..

## 定义自定义模型流水线，使用来自`google_cloud_pipeline_components` 的组件

接下来，您可以定义流水线。

`experimental.run_as_aiplatform_custom_job` 方法接受之前定义的组件以及`worker_pool_specs`列表（在这种情况下是一个）作为参数。自定义训练作业将使用这个列表进行配置。

然后，使用[`google_cloud_pipeline_components`](https://github.com/kubeflow/pipelines/tree/master/components/google-cloud) 组件来定义流水线的其余部分：上传模型，创建端点，以及将模型部署到端点。

*注意：* 虽然这个例子中没有展示，但模型部署将会在没有提供端点的情况下创建一个端点。

In [ ]:
WORKER_POOL_SPECS = [
    {
        "containerSpec": {
            "args": TRAINER_ARGS,
            "env": [{"name": "AIP_MODEL_DIR", "value": WORKING_DIR}],
            "imageUri": TRAIN_IMAGE,
        },
        "replicaCount": "1",
        "machineSpec": {
            "machineType": TRAIN_COMPUTE,
            "accelerator_type": TRAIN_TPU,
            "accelerator_count": TRAIN_NTPU,
        },
    }
]

定义管道

该管道有四个主要步骤：

1) run_as_experimental_custom_job 运行将执行训练任务的docker容器

2) ModelUploadOp 将训练好的模型上传到 Vertex

3) EndpointCreateOp 创建模型端点

4) 最后，ModelDeployOp 部署模型到端点

In [ ]:
@kfp.dsl.pipeline(name="train-endpoint-deploy")
def pipeline(
    project: str = PROJECT_ID,
    model_display_name: str = MODEL_DISPLAY_NAME,
    serving_container_image_uri: str = DEPLOY_IMAGE,
):

    custom_job_task = CustomTrainingJobOp(
        display_name="tpu model training",
        worker_pool_specs=WORKER_POOL_SPECS,
    )

    import_unmanaged_model_task = importer_node.importer(
        artifact_uri=WORKING_DIR,
        artifact_class=artifact_types.UnmanagedContainerModel,
        metadata={
            "containerSpec": {
                "imageUri": serving_container_image_uri  # "us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-9:latest",
            },
        },
    ).after(custom_job_task)

    model_upload_op = ModelUploadOp(
        project=project,
        display_name=model_display_name,
        unmanaged_container_model=import_unmanaged_model_task.outputs["artifact"],
    )

    endpoint_create_op = EndpointCreateOp(
        project=project,
        display_name="tpu-pipeline-created-endpoint",
    )

    _ = ModelDeployOp(
        endpoint=endpoint_create_op.outputs["endpoint"],
        model=model_upload_op.outputs["model"],
        deployed_model_display_name=model_display_name,
        dedicated_resources_machine_type=DEPLOY_COMPUTE,
        dedicated_resources_min_replica_count=1,
        dedicated_resources_max_replica_count=1,
        dedicated_resources_accelerator_type=DEPLOY_GPU.name,
        dedicated_resources_accelerator_count=DEPLOY_NGPU,
    )

## 编译管道

接下来，编译管道。

In [ ]:
compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path="tpu_train_cifar10_pipeline.json",
)

运行管道

接下来，运行管道。

In [ ]:
DISPLAY_NAME = "tpu_cifar10_training"

job = aip.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="tpu_train_cifar10_pipeline.json",
    pipeline_root=PIPELINE_ROOT,
)

job.run()

! rm tpu_train_cifar10_pipeline.json

点击生成的链接，以在云控制台中查看你的运行。

在用户界面中，当你点击它们时，许多流水线DAG节点会展开或折叠。

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除您在本教程中创建的各个资源 -- *注意:* 这是自动生成的，不是所有资源可能适用于此教程：
### 获取清理的管道资源
用于获取任务详情的函数

In [ ]:
def get_task_detail(
    task_details: List[Dict[str, Any]], task_name: str
) -> List[Dict[str, Any]]:
    for task_detail in task_details:
        if task_detail.task_name == task_name:
            return task_detail

In [ ]:
pipeline_task_details = (
    job.gca_resource.job_detail.task_details
)  # fetch pipeline task details


# fetch endpoint from pipeline and delete the endpoint
endpoint_task = get_task_detail(pipeline_task_details, "endpoint-create")
endpoint_resourceName = (
    endpoint_task.outputs["endpoint"].artifacts[0].metadata["resourceName"]
)
endpoint = aip.Endpoint(endpoint_resourceName)
# undeploy model from endpoint
endpoint.undeploy_all()
endpoint.delete()

# fetch model from pipeline and delete the model
model_task = get_task_detail(pipeline_task_details, "model-upload")
model_resourceName = model_task.outputs["model"].artifacts[0].metadata["resourceName"]
model = aip.Model(model_resourceName)
model.delete()

job.delete()

In [ ]:
# Warning: Setting this to true will delete everything in your bucket
delete_bucket = False


if delete_bucket and "BUCKET_URI" in globals():
    ! gsutil rm -r $BUCKET_URI